In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os.path


import seaborn as sns
from scipy import stats


# Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib.pyplot import *
import imageio
from skimage.io import imread

from tqdm import tqdm

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
import sys
#from natsort import natsorted
sns.set()

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Sklearn Liraries
import sklearn
from sklearn import preprocessing

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline


import sklearn
from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of data
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
print(sklearn.__version__)

1.2.1


In [3]:
#!conda install -c conda-forge scikit-learn=0.23.2
#!pip install --pre pycaret

In [4]:
#set directory for csv files
dir_name = r'C:\Users\vasan\OneDrive\EmissionsDatathon\Zeroing Methane Emissions - Dataset'
filename_suffix = 'xls'

In [6]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
df = pd.read_excel("dftif2.xlsx", thousands=',', skiprows = skiprows)
df.head()

,Unnamed: 0,Red,Green,Blue,Name,qplume
0,0,88.579944,102.179257,171.641196,ang20190922t192642-2_r4578_c217_ctr.tif,2579.528100
1,1,80.060855,94.922414,97.609589,ang20190923t163307-G_r4113_c198_ctr.tif,934.430300
2,2,62.973427,82.282730,115.447802,ang20190926t223557-D_r4139_c225_ctr.tif,443.098484
3,3,98.197928,132.865953,134.486978,ang20191006t150453-6_r4168_c237_ctr.tif,1032.395030
4,4,80.098039,113.231552,113.058824,ang20191007t182352-2_r4062_c150_ctr.tif,826.823256


In [7]:
df.shape

(5592, 6)

In [8]:
df = df.drop('Unnamed: 0', axis=1)

In [9]:
df = df.drop('Name', axis=1)

In [10]:
df.head()

,Red,Green,Blue,qplume
0,88.579944,102.179257,171.641196,2579.528100
1,80.060855,94.922414,97.609589,934.430300
2,62.973427,82.282730,115.447802,443.098484
3,98.197928,132.865953,134.486978,1032.395030
4,80.098039,113.231552,113.058824,826.823256


In [11]:
df = df.astype(float)

In [12]:
np.isnan(df).any()

Red       False
Green     False
Blue      False
qplume    False
dtype: bool

In [13]:
df.isna().sum()

Red       0
Green     0
Blue      0
qplume    0
dtype: int64

In [14]:
df = df.dropna()

In [15]:
df.shape

(5592, 4)

In [16]:
print("Upper limit",df['qplume'].mean() + 3*df['qplume'].std())
print("Lower limit",df['qplume'].mean() - 3*df['qplume'].std())

Upper limit 2753.399332254686
Lower limit -1707.5576973313132


In [17]:
#df = df[(df['qplume'] < 2753.399332254686)]
#df.shape

In [18]:
# Creates a mask where values that are true go into the training/test set
# Note that I done it so that the random number is predictable

msk = np.random.seed(0)
msk = np.random.rand(len(df))<0.8

raw_train_validate_set = df[msk]
raw_test_set = df[~msk]

print(raw_train_validate_set.shape)
print(raw_test_set.shape)

(4486, 4)
(1106, 4)


In [19]:
raw_train_validate_set.to_excel(r'dftrain.xlsx', index = False, header=True)
raw_test_set.to_excel(r'dftest.xlsx', index = False, header=True)

In [20]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
dftrain = pd.read_excel("dftrain.xlsx", thousands=',', skiprows = skiprows)
dftrain = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
dftrain.head()

,Red,Green,Blue,qplume
0,88.579944,102.179257,171.641196,2579.528100
1,80.060855,94.922414,97.609589,934.430300
2,62.973427,82.282730,115.447802,443.098484
3,98.197928,132.865953,134.486978,1032.395030
4,80.098039,113.231552,113.058824,826.823256


In [21]:
target_columns = [
    "qplume"
]

y_train = dftrain[target_columns]
X_train = dftrain.drop(columns=target_columns)

In [22]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [23]:
y_predtrain = model_lr.predict(X_train)

In [24]:
print('Coefficients:', model_lr.coef_)
print('Intercept:', model_lr.intercept_)

Coefficients: [[15.90548932  6.56580362  0.6240231 ]]
Intercept: [-1162.59382599]


In [25]:
# Calculate the mean squared error
mse = mean_squared_error(y_train, y_predtrain)

# Calculate the root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

r2 = r2_score(y_train, y_predtrain)
print("Test R^2 Score: {:.5f}".format(r2))

Root Mean Squared Error: 640.7481345399855
Test R^2 Score: 0.25715


In [26]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
dftest = pd.read_excel("dftest.xlsx", thousands=',', skiprows = skiprows)
dftest = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
dftest.head()

,Red,Green,Blue,qplume
0,88.579944,102.179257,171.641196,2579.528100
1,80.060855,94.922414,97.609589,934.430300
2,62.973427,82.282730,115.447802,443.098484
3,98.197928,132.865953,134.486978,1032.395030
4,80.098039,113.231552,113.058824,826.823256


In [27]:
y_test = dftest[target_columns]
X_test = dftest.drop(columns=target_columns)

In [28]:
y_predtest = model_lr.predict(X_test)

In [29]:
# Calculate the mean squared error
mse = mean_squared_error(y_train, y_predtest)

# Calculate the root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

r2 = r2_score(y_train, y_predtest)
print("Test R^2 Score: {:.5f}".format(r2))

Root Mean Squared Error: 640.7481345399855
Test R^2 Score: 0.25715


In [33]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
dsub = pd.read_excel("dsub.xlsx", thousands=',', skiprows = skiprows)
dsub = dsub.loc[:, ~dsub.columns.str.contains('^Unnamed')] 
dsub.head()

,Red,Green,Blue
0,115.077159,108.672492,82.528371
1,65.539035,89.165645,124.884486
2,63.963010,81.884271,112.636710
3,47.245902,47.245902,100.546875
4,50.625000,50.625000,105.434783


In [34]:
dsub.shape

(20, 3)

In [35]:
y_predsub = model_lr.predict(dsub)

In [36]:
y_predsub

array([[1432.78654597],
       [ 543.21151142],
       [ 462.69309655],
       [ -38.17375684],
       [  40.80911958],
       [ 137.16104777],
       [ 336.72828605],
       [ 528.08960479],
       [ 780.17774876],
       [ 185.07689507],
       [ 512.75944411],
       [ 213.81699051],
       [ 709.18419446],
       [ 349.77327502],
       [ 299.71985578],
       [ 479.19548001],
       [ 490.27819837],
       [ 332.74092074],
       [ 501.6332662 ],
       [ 645.55835547]])

In [66]:
np.savetxt('13.csv', y_predsub, delimiter=',')